In [151]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [152]:
train=pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [153]:
plt.style.use(style='ggplot')
plt.rcParams['figure.figsize'] = (12,8)

In [154]:
target=np.log(train.SalePrice)

In [155]:
numeric_features = train.select_dtypes(include=[np.number])
corr = numeric_features.corr()

In [156]:
train= train[train['GarageArea'] <1200]

In [157]:
nulls=pd.DataFrame(train.isnull().sum().sort_values(ascending=False)[:25])
nulls.columns = ['Null Count']
nulls.index.name='Feature'

In [158]:
categoricals=train.select_dtypes(exclude=[np.number])

In [159]:
train['enc_street'] = pd.get_dummies(train.Street, drop_first=True)
test['enc_street'] = pd.get_dummies(test.Street, drop_first=True)

In [160]:
def encode(x): return 1 if x=='Partial' else 0
train['enc_condition'] = train.SaleCondition.apply(encode)
test['enc_condition'] = test.SaleCondition.apply(encode)

In [161]:
condition_pivot = train.pivot_table(index='SaleCondition', values='SalePrice', aggfunc=np.median)

In [162]:
data = train.select_dtypes(include=[np.number]).interpolate().dropna()

In [163]:
y=np.log(train.SalePrice)
X=data.drop(['SalePrice', 'Id'], axis=1)

In [164]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.33)

In [165]:
linreg = linear_model.LinearRegression()
model = linreg.fit(X_train, y_train)

In [166]:
model.score(X_test, y_test)

0.8882477709262531

In [167]:
predictions = model.predict(X_test)
mean_squared_error(y_test, predictions)

0.01784179451956794

In [168]:
actual_values = y_test

In [169]:
submission = pd.DataFrame()
# the firs column must contain the ID from the test data
submission['Id'] = test.Id

feats = test.select_dtypes(
include = [np.number]).drop(['Id'], axis=1).interpolate()

predictions = model.predict(feats)
predictions[:10]


array([11.76725362, 11.71929504, 12.07656074, 12.20632678, 12.11217655,
       12.05709882, 12.16036698, 12.01665734, 12.17126892, 11.66318882])

In [170]:
final_predictions = np.exp(predictions)
final_predictions[:10]

array([128959.49172585, 122920.74024358, 175704.82598102, 200050.83263755,
       182075.46986405, 172318.33397533, 191064.621642  , 165488.55901671,
       193158.99133191, 116214.02546462])

In [171]:
submission['SalePrice'] = final_predictions

In [172]:
submission.to_csv('submission_3.csv', index=False)